In [4]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs


In [96]:
gurobi_env = Gurobi.Env()
to = TimerOutput()
myFile = "./SmallExample.jl"
include(myFile)
myFile = "./shortestPath.jl"
include(myFile)


arcs = [1 2; 1 5; 2 3; 2 4; 3 6; 4 7; 5 4; 7 8; 7 10]
A = length(arcs[:,1])
c = ones(length(arcs))
b = 2 
# p = ones(T)./T
source = 1
sink = 10
T = [6,7]
scenNum = length(T)
p = ones(Float64,scenNum)./scenNum
v = [10,20]

Academic license - for non-commercial use only


2-element Array{Int64,1}:
 10
 20

In [97]:
destination = sink
origin = source
x_now = zeros(length(arcs))
y, gx, tree, path, y_Set = gx_bound(arcs, A, c, origin, x_now,T,scenNum)

1


([1, 0, 0, 1, 0, 1, 0, 0, 1], 4.0, [1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 2, 4, 7, 10], [1.0 0.0 … 0.0 0.0; 1.0 1.0 … 1.0 1.0])

In [98]:
println(arcs[:,1][tree.>0])
println(arcs[:,2][tree.>0])

println(y_Set)

[1, 1, 2, 2, 3, 4, 7, 7]
[2, 5, 3, 4, 6, 7, 8, 10]
[1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0; 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0]


In [99]:
#Solve the 2-stage problem: Full formulation:
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
m = Model(() -> Gurobi.Optimizer(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 


# @variable(m, w >=0)
@variable(m,  x1[1:A], Bin)
@variable(m, x2[1:scenNum,1:A], Bin)
# @variable(m, 1e6 >= z[1:200000] >= 0)
@variable(m, z[1:scenNum] >=0)
@variable(m, 1>= r[1:scenNum] >= 0)
# constr = Array{JuMP.ConstraintRef}(undef, 200000)

# @constraint(m, sum(x[i] for i=1:A) <= b) #attack budget = 2
# constr[1] = @constraint(m, w <= SP_init + sum(yy[i]*x[i]*d[i] for i=1:Len) + z[1]  )
for k = 1:scenNum
    @constraint(m, sum(x2[k,i] for i=1:A) <= b - sum(x1[i] for i =1:A))
end
@objective(m, Min, sum(p[i]*z[i] for i = 1:length(p)) )


for k = 1:scenNum
    @constraint(m, (sum(y_Set[k,j]*(x1[j]+x2[k,j]) for j = 1:A if arcs[j,1]!=source)) >= r[k] ) 
    @constraint(m, z[k] >= v[k]*(1-r[k]))
end

optimize!(m)

println(m)


Min 0.5 z[1] + 0.5 z[2]
Subject to
 x1[4] + x1[6] + x2[1,4] + x2[1,6] - r[1] >= 0.0
 z[1] + 10 r[1] >= 10.0
 x1[3] + x1[4] + x1[5] + x1[6] + x1[7] + x1[8] + x1[9] + x2[2,3] + x2[2,4] + x2[2,5] + x2[2,6] + x2[2,7] + x2[2,8] + x2[2,9] - r[2] >= 0.0
 z[2] + 20 r[2] >= 20.0
 x1[1] + x1[2] + x1[3] + x1[4] + x1[5] + x1[6] + x1[7] + x1[8] + x1[9] + x2[1,1] + x2[1,2] + x2[1,3] + x2[1,4] + x2[1,5] + x2[1,6] + x2[1,7] + x2[1,8] + x2[1,9] <= 2.0
 x1[1] + x1[2] + x1[3] + x1[4] + x1[5] + x1[6] + x1[7] + x1[8] + x1[9] + x2[2,1] + x2[2,2] + x2[2,3] + x2[2,4] + x2[2,5] + x2[2,6] + x2[2,7] + x2[2,8] + x2[2,9] <= 2.0
 z[1] >= 0.0
 z[2] >= 0.0
 r[1] >= 0.0
 r[2] >= 0.0
 r[1] <= 1.0
 r[2] <= 1.0
 x1[1] binary
 x1[2] binary
 x1[3] binary
 x1[4] binary
 x1[5] binary
 x1[6] binary
 x1[7] binary
 x1[8] binary
 x1[9] binary
 x2[1,1] binary
 x2[2,1] binary
 x2[1,2] binary
 x2[2,2] binary
 x2[1,3] binary
 x2[2,3] binary
 x2[1,4] binary
 x2[2,4] binary
 x2[1,5] binary
 x2[2,5] binary
 x2[1,6] binary
 x2[2,6] bina

In [100]:
println("MP = ", JuMP.objective_value.(m))
println("x1 = ", findall(JuMP.value.(x1).==1))
for k = 1:scenNum
    println("x2,",k," = ", findall(JuMP.value.(x2[k,:]).==1))
    println
end

MP = 0.0
x1 = Int64[]
x2,1 = [4, 6]
x2,2 = [4, 8]


In [47]:
y_Set = Array{Int64, (scenNum, A)}
println(y_Set)
# y_Set[1,:] = ones(16)

Array{Int64,(4, 16)}
